In [1]:
import cv2 as cv
import imutils
import numpy as np
import sys
import os
import torch
import torchvision
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data
import torchvision.datasets as dset
import torchvision.utils as vutils
from torch.utils.data import DataLoader
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import random
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
import pyautogui

In [2]:
def perform(action):
    if (action=='palmdown'):
        pyautogui.press(['down', 'down', 'down', 'down', 'down'])
    
    elif (action=='scrollup'):
        pyautogui.press(['up', 'up', 'up', 'up', 'up'])
        
    elif (action=='fingerright'):
        pyautogui.hotkey('ctrl', 'right')
        
    elif (action=='palmleft'):
        pyautogui.hotkey('ctrl', 'left')
        
    elif (action=='close'):
        pyautogui.hotkey('space')
        
    elif (action=='fistround'):
        pyautogui.press('m')

In [3]:
#Using GPU
print(sys.version)
device = 'cuda'
#Checking for GPU
print(torch.version.cuda)
print(torch.cuda.device_count())
print(torch.cuda.is_available())
torch.cuda.empty_cache() 

3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
10.2
1
True


In [4]:
lookup = dict()
reverselookup = dict()

count = 0

for j in os.listdir('./dynamicdata/'):
    if not j.startswith('ges'):
        lookup[j] = count
        reverselookup[count] = j
        count += 1
        
print(lookup)
print(reverselookup)

{'close': 0, 'fingerright': 1, 'fistround': 2, 'palmdown': 3, 'palmleft': 4, 'scrollup': 5}
{0: 'close', 1: 'fingerright', 2: 'fistround', 3: 'palmdown', 4: 'palmleft', 5: 'scrollup'}


In [5]:
class C3D(nn.Module):
  """
  The C3D network as described in [1]
  Batch Normalization as described in [2]

  """

  def __init__(self):
    super(C3D, self).__init__()

    self.conv1 = nn.Conv3d(1, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
    #self.conv1_bn = nn.BatchNorm3d(64)
    self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

    self.conv2 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
    #self.conv2_bn = nn.BatchNorm3d(128)
    self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

    self.conv3a = nn.Conv3d(128, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
    #self.conv3a_bn = nn.BatchNorm3d(256)
    self.conv3b = nn.Conv3d(128, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
    #self.conv3b_bn = nn.BatchNorm3d(256)
    self.pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

    self.conv4a = nn.Conv3d(128, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
    #self.conv4a_bn = nn.BatchNorm3d(512)
    self.conv4b = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
    #self.conv4b_bn = nn.BatchNorm3d(512)
    self.pool4 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

    self.conv5a = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
    #self.conv5a_bn = nn.BatchNorm3d(512)
    self.conv5b = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
    #self.conv5b_bn = nn.BatchNorm3d(512)
    self.pool5 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(0, 1, 1))

    self.fc6 = nn.Linear(4096, 1024)
    #self.fc7 = nn.Linear(1024, 1024)
    self.fc8 = nn.Linear(1024, num_classes)
    self.relu = nn.ReLU()
    self.logsoftmax = nn.Softmax(dim=1)

  def forward(self, x):

    h = x.view(-1, 1, 16, 112, 112)

    h = self.relu(self.conv1(h))
    h = self.pool1(h)

    h = self.relu(self.conv2(h))
    h = self.pool2(h)

    h = self.relu(self.conv3a(h))
    h = self.relu(self.conv3b(h))
    h = self.pool3(h)

    h = self.relu(self.conv4a(h))
    h = self.relu(self.conv4b(h))
    h = self.pool4(h)

    h = self.relu(self.conv5a(h))
    h = self.relu(self.conv5b(h))
    h = self.pool5(h)

    h = h.view(-1, 4096)
    h = self.relu(self.fc6(h))
    #h = self.relu(self.fc7(h))
    h = self.fc8(h)
    h = self.logsoftmax(h)
    return h

"""
References
----------
[1] Tran, Du, et al. "Learning spatiotemporal features with 3d convolutional networks." 
Proceedings of the IEEE international conference on computer vision. 2015.
[2] Ioffe, Surgey, et al. "Batch Normalization: Accelerating deep network training 
by reducing internal covariate shift."
arXiv:1502.03167v2 [cs.LG] 13 Feb 2015
"""

'\nReferences\n----------\n[1] Tran, Du, et al. "Learning spatiotemporal features with 3d convolutional networks." \nProceedings of the IEEE international conference on computer vision. 2015.\n[2] Ioffe, Surgey, et al. "Batch Normalization: Accelerating deep network training \nby reducing internal covariate shift."\narXiv:1502.03167v2 [cs.LG] 13 Feb 2015\n'

In [6]:
model = torch.load('./models/c3d1.pth')

C:\Users\Aakriti\anaconda3\lib\site-packages\torch\serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.parallel.data_parallel.DataParallel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Users\Aakriti\anaconda3\lib\site-packages\torch\serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv3d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Users\Aakriti\anaconda3\lib\site-packages\torch\serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's s

In [7]:
image_size = 112

def recognize_gest(): 
    path = './dynamicdata/gesty/'
    gest = None
    frame_count = 16
    transform = transforms.Compose([
                transforms.Resize(size=image_size),  # Image net standards
                transforms.ToTensor(), 
                transforms.Normalize((0.5), (0.5))
                ])
    
    for j in range(frame_count):
        img = Image.open(path+str(j+1)+'.jpg')
        #print(img.shape)c
        img = transform(img)
        img = img.view(-1, image_size, image_size, 1)

        if gest is None:
            gest = img
        else:
            gest = torch.cat((gest, img), dim=3)
    print('Concatnated!', end=' ')
    
    with torch.no_grad():
        gest = gest.view(16, 1, image_size, image_size)
        gest = gest.to(device)#Passing the entire test set to the GPU
        test_out = model(gest.float()).cpu()
        test_pred = torch.argmax(test_out, dim = 1, keepdim=True).cpu().numpy()
        #test_pred_np = test_pred.cpu().numpy() 
        test_pred = np.reshape(test_pred, test_pred.shape[0])
        action = reverselookup[test_pred[0]]
        print(action)
        perform(action)

In [8]:
def run_avg(image, aWeight):
    global bg
    if bg is None:
        bg = image.astype('float')
        
        return
    cv.accumulateWeighted(image, bg, aWeight)
    #print(bg)

In [9]:
def segment (image, threshold=18):
    global bg
    #cv.imshow('back', np.uint8(bg))    
    #cv.imshow('Handy', image)
    diff = cv.absdiff(np.uint8(bg), image)
    #cv.imshow('diff', diff)
    
    blur = cv.GaussianBlur(diff, (3, 3), 0)
    blur = cv.medianBlur(diff, 11)
    
    #thresholded = cv.adaptiveThreshold(blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 5, 12)
    thresholded = cv.threshold(diff, threshold, 255, cv.THRESH_BINARY)[1]
    disc = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
    disc1 = cv.getStructuringElement(cv.MORPH_ELLIPSE, (2, 2))
    thresholded = cv.dilate(thresholded, disc, iterations =1)
    thresholded = cv.morphologyEx(thresholded, cv.MORPH_CLOSE, disc, iterations =5)
    
    (cnts, _) = cv.findContours(thresholded.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    if len(cnts) == 0:
        return
    else:
        segmented = max(cnts, key = cv.contourArea)
        
        tot_pixel = np.count_nonzero(thresholded)
        if tot_pixel>3000 :
            return (thresholded, segmented)
        else:
            return None


In [10]:
def hist_skin(hist, hsv):
    
    dst = cv.calcBackProject([hsv], [0, 1], hist, [0, 200, 0, 256], 1)
    dst1 = dst.copy()
    disc = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
    cv.filter2D(dst, -1, disc, dst)
    blur = cv.GaussianBlur(dst, (11, 11), 0)
    blur = cv.medianBlur(blur, 15)
    #cv.imshow("Blur", blur)
    ret, thresh = cv.threshold(blur, 0, 255, cv.THRESH_BINARY)#+cv.THRESH_OTSU)
    #thresh = cv.merge((thresh, thresh, thresh))
    return thresh

In [11]:
def gest_avg(cap, gest, aWeight=0.5):
    if gest is None:
        gest = cap.astype('float')
        return gest
    return cv.accumulateWeighted(cap, gest, aWeight)
    #print(bg)

In [16]:
bg = None
if __name__ == '__main__':
    
    aWeight = 0.5
    
    flag = False
    
    capture = False
    
    camera = cv.VideoCapture(0)
    
    top, right, bottom, left = 10, 290, 266, 546
    
    in_top, in_right, in_bottom, in_left = top+50, right+100, bottom-50, left-100
    
    num_frames = 0
    
    gest_frames = 0
    
    gest = None
    
    hand = None
    
    dim = (50, 50)
    
    while True:
        (ret, frame) = camera.read()
        
        frame = cv.resize(frame, (640, 480))
        frame = cv.flip(frame, 1)
        
        clone = frame.copy()
        
        (height, width) = frame.shape[:2]
        roi = frame[top:bottom, right:left]
        hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
        
        in_roi = frame[in_top:in_bottom, in_right:in_left]
        
        gray = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
        gray = cv.GaussianBlur(gray, (3, 3), 0)
        gray = cv.medianBlur(gray, 3)
        
        keypress = cv.waitKey(1) & 0xFF
        
        if keypress == ord('c'):
            break
            
        elif keypress == ord('s'):
            flag = True
            print('skin histogram captured!')
            hsvCrop = cv.cvtColor(in_roi, cv.COLOR_BGR2HSV)
            cv.imshow("i", hsvCrop)
            hist = cv.calcHist([hsvCrop], [0, 1], None, [200, 256], [0, 200, 0, 256])
            cv.normalize(hist, hist, 0, 255, cv.NORM_MINMAX)
            
        elif hand is not None:
            if capture is False:
                #cv.waitKey(5)
                capture = True
                gest_frames = 0
                #print('g')
            
        if keypress == 13:
            bg = None
            num_frames = 0
        
        if num_frames < 100:
            run_avg(gray, aWeight)
        else:
            if(num_frames==100):
                print('background captured!')
            hand = segment(gray)
            
            if hand is not None:
                (thresholded, segmented) = hand
                
                cv.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                if flag:
                    skin = hist_skin(hist, hsv_roi)
                    #print(thresholded.shape, skin.shape)
                    thresholded = cv.bitwise_or(thresholded, skin)
                if capture:
                    if (gest_frames == 0):
                        gesty = []
                    gest = gest_avg(thresholded, gest)
                    if (gest_frames+1)%2 ==0:
                        print(gest_frames, end=' ')
                        #gesty.append[gest]
                        cv.imwrite('./dynamicdata/gesty/' + str(int((gest_frames+1)/2)) + '.jpg', gest)
                        gest = None
                        #out.write(thresholded)
                    gest_frames += 1
                    if gest_frames > 31:
                        if gest_frames == 32:
                            print('Gesture captured!', end=' ')
                            recognize_gest()
                        if gest_frames == 80:
                            capture = False
                            #cv.waitKey(1500)
                            
            elif gest_frames >= 32:
                capture = False
                    
                cv.imshow('Thresholded', thresholded)
                
                
        cv.rectangle(clone, (left, top), (right, bottom), (0, 255, 0), 2)
        cv.rectangle(clone, (in_left, in_top), (in_right, in_bottom), (255, 0, 0), 1)
        
        num_frames += 1
        cv.imshow('Video Feed', clone)
            
cv.destroyAllWindows()
camera.release()

background captured!
background captured!
1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! scrollup
1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! fistround
33 35 37 39 41 43 45 47 49 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! fistround
33 35 37 39 41 43 45 47 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! scrollup
1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! close
33 35 37 39 41 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! close
33 35 37 39 41 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! close
33 35 37 39 41 43 45 47 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! fingerright
33 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! fingerright
33 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! fistround
1 3 5 7 9 11 13 1

33 35 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! scrollup
33 35 37 39 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! scrollup
33 35 37 39 41 43 45 47 background captured!
1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! close
33 35 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! close
33 35 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! close
33 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! close
33 35 37 39 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! close
33 35 37 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! fingerright
33 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! fingerright
1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! palmleft
33 35 37 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatn

33 35 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! close
33 35 37 39 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! close
33 35 37 39 41 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! fingerright
1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! fingerright
1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! palmleft
33 35 37 39 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! palmleft
33 35 37 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! scrollup
33 35 37 39 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! scrollup
1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! scrollup
1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! palmdown
33 35 37 39 41 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 Gesture captured! Concatnated! palmdown
33 35 